In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Add
import os
from keras.layers import LeakyReLU
from tqdm import tqdm

2023-11-09 14:30:49.514238: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-09 14:30:49.800767: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 14:30:49.800809: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 14:30:49.801970: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 14:30:49.918143: I tensorflow/core/platform/cpu_feature_g

In [2]:
def load_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image)
    image = tf.image.resize(image, (64, 64))
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    image = tf.reshape(image, shape=(64, 64, 3))
    #image = tf.expand_dims(image, axis=0)

    return image

path = '../data/'
images = [os.path.join(path, image) for image in os.listdir(path)]

batch_size = 125

dataset = tf.data.Dataset.from_tensor_slices((images))
dataset = dataset.map(load_image)
dataset = dataset.shuffle(1000).batch(batch_size)

#
# Model

#### Version 1

In [3]:
input_shape = (94, 94, 3)
latent_dim = 32


class Encoder(Model):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=input_shape),
            layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((3, 3), padding='same'),
            layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((3, 3), padding='same'),
            layers.Flatten()
        ])
        self.mean = layers.Dense(latent_dim)
        self.log_var = layers.Dense(latent_dim)
    
    def call(self, X):
        X = self.encoder(X)
        return self.mean(X), self.log_var(X)


class Decoder(Model):
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(1, latent_dim)),
            layers.Dense(7 * 7 * 64, activation='relu'),
            layers.Reshape((7, 7, 64)),
            layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same'),
            layers.UpSampling2D((2,2)),
            layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same'),
            layers.UpSampling2D((2,2)),
            layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')
        ])
    
    def call(self, X):
        return self.decoder(X)


class VAE(Model):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)

    def call(self, X):
        mu, sigma = self.encoder(X)
        epsilon = tf.random.normal(tf.shape(mu))
        z = tf.expand_dims(mu + epsilon * sigma, axis=0)
        X_reconstructed = self.decoder(z)

        return X_reconstructed, mu, sigma

path = '../data/'
images = os.listdir(path)
image = load_image(path + images[1])

latent_dim = 32

model = VAE(latent_dim)
output = (model(image)[0] * 255.0).numpy().astype(int)

print(output.shape)
        

ValueError: Exception encountered when calling layer 'encoder' (type Encoder).

Input 0 of layer "sequential" is incompatible with the layer: expected shape=(None, 94, 94, 3), found shape=(64, 64, 3)

Call arguments received by layer 'encoder' (type Encoder):
  • X=tf.Tensor(shape=(64, 64, 3), dtype=float32)

#### Version 2

In [3]:
latent_dim = 32

class VAE(Model):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            layers.InputLayer((368, 640, 3)),
            layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
            layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            layers.Flatten(),
            layers.Dense(latent_dim*2, activation='selu')
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(1024, activation='selu', input_shape=(latent_dim,)),
            layers.Dense(23 * 40 * 512, activation='selu'),
            layers.Reshape((23, 40, 512)),
            layers.Conv2DTranspose(256, (5, 5), activation=LeakyReLU(0.02), strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.Conv2DTranspose(128, (5, 5), activation=LeakyReLU(0.02), strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.Conv2DTranspose(64, (5, 5), activation=LeakyReLU(0.02), strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.Conv2DTranspose(32, (5, 5), activation=LeakyReLU(0.02), strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.Conv2DTranspose(3, (5, 5), activation='sigmoid', strides=1, padding='same'),
            layers.BatchNormalization()
        ])

    def call(self, X):
        encoded = self.encoder(X)
        mu, sigma = tf.split(encoded, num_or_size_splits=2, axis=-1)
        epsilon = tf.random.normal(tf.shape(sigma))
        z = mu + epsilon * tf.exp(0.5 * sigma)
        reconstruction = self.decoder(z)
        return reconstruction

model = VAE(latent_dim)

: 

In [5]:
image = cv2.imread('./eucalipto.png')

model(image)

ValueError: Exception encountered when calling layer 'vae_1' (type VAE).

Input 0 of layer "sequential_2" is incompatible with the layer: expected shape=(None, 368, 640, 3), found shape=(368, 640, 3)

Call arguments received by layer 'vae_1' (type VAE):
  • X=tf.Tensor(shape=(368, 640, 3), dtype=uint8)

In [20]:
def reconstruction_loss(y, pred):
    return tf.reduce_mean(tf.square(y - pred))

In [22]:
mse_losses = []

# 368x640

for epoch in tqdm(range(3)):
    for batch in dataset:
        with tf.GradientTape() as tape:
            pred = tf.reshape(model(batch), shape=[-1])
            y = tf.reshape(batch, shape=[-1])

            mse = reconstruction_loss(y, pred)
            mse_losses.append(mse.numpy())

  0%|          | 0/3 [00:00<?, ?it/s]

tf.Tensor(0.096620776, shape=(), dtype=float32)
tf.Tensor(0.094599545, shape=(), dtype=float32)
tf.Tensor(0.09532998, shape=(), dtype=float32)
tf.Tensor(0.09585252, shape=(), dtype=float32)


 33%|███▎      | 1/3 [00:05<00:10,  5.10s/it]

tf.Tensor(0.097649045, shape=(), dtype=float32)
tf.Tensor(0.094190575, shape=(), dtype=float32)
tf.Tensor(0.09459026, shape=(), dtype=float32)


 67%|██████▋   | 2/3 [00:07<00:03,  3.78s/it]

tf.Tensor(0.09597202, shape=(), dtype=float32)
tf.Tensor(0.096577436, shape=(), dtype=float32)
tf.Tensor(0.095426545, shape=(), dtype=float32)
tf.Tensor(0.095999196, shape=(), dtype=float32)


100%|██████████| 3/3 [00:10<00:00,  3.60s/it]

tf.Tensor(0.09440139, shape=(), dtype=float32)
